In [42]:
from importlib import reload
import folium
import pandas as pd
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
import src.localization as localization
reload(localization)
from src.localization import read_data
from src.localization import get_distance_in_meters
from src.localization import gen_regressors
from src.localization import gen_fingerprints
positions, df = read_data('resources/data/LocTreino_Equipe_2.csv')
bts_positions, df_btss = read_data('resources/data/dados_BTSs.csv')
center_lat = np.mean([lat for lat in bts_positions[:, 0]])
center_lon = np.mean([lon for lon in bts_positions[:, 1]])
original_pts_map = folium.Map(location=[center_lat, center_lon],zoom_start=15)

icon_c0 = folium.Icon(icon='feed', iconColor='white', prefix='fa')
icon_c1 = folium.Icon(icon='feed', iconColor='white', prefix='fa')
# Plot BTSs locations
#for location in bts_positions:
#    marker = folium.Marker(location=location)
#    marker.add_to(original_pts_map)
location = bts_positions[0]
marker = folium.Marker(location=location,tooltip='sport',popup='Erb 1', icon=icon_c0)
marker.add_to(original_pts_map)

location = bts_positions[1]
marker = folium.Marker(location=location,tooltip='sport',popup='Erb 2', icon=icon_c1)
marker.add_to(original_pts_map)

# Export map
original_pts_map.save("resources/maps/original_pts_map_ta_triangulation.html")

# Show map
original_pts_map

In [43]:
fingerprint_cols = ['pathBTS1',
                    'pathBTS2',
                    'pathBTS3',
                    'pathBTS4',
                    'pathBTS5',
                    'pathBTS6']
targets = [df[t_c] for t_c in fingerprint_cols]
#targets = [df_points_all[t_c] for t_c in fingerprint_cols]

# Input features
position_columns = ['lat', 'lon']
input_features = df[position_columns].values
#input_features = df_points_all[position_columns].values
regs = gen_regressors(input_features,
                      targets)

RESOLUTION = 20 #meters

print("SQUARE GRID GENERATOR")
print("\tResolution: {} m".format(RESOLUTION))



# Calculate limits
lats = np.array(list(df['lat']))
lons = np.array(list(df['lon']))
max_lat = lats.max()
min_lat = lats.min()
max_lon = lons.max()
min_lon = lons.min()

diff_lat = 0.0
diff_lon = 0.0
inc = 0.0000001
ref_lat = min_lat
ref_lon = min_lon
while(get_distance_in_meters(ref_lat, ref_lon, ref_lat+diff_lat, ref_lon) < RESOLUTION):
    diff_lat += inc
while(get_distance_in_meters(ref_lat, ref_lon, ref_lat, ref_lon+diff_lon) < RESOLUTION):
    diff_lon += inc

#while(get_distance_in_meters(max_lat, max_lon,min_lat,max_lon) < resolution)
#    diff_lat += inc

#vertical_distance = get_distance_in_meters(max_lat, max_lon,max_lat,min_lon)
#print(vertical_distance)

# Generate fingerprints for squared grid
cells = [] #added TO FIX
print(max_lat)
print(min_lat)

# Add grid points
number_of_squares = 20 #old value 20

#grid_lats = np.arange(min_lat, max_lat, diff_lat)

#grid_lons = np.arange(min_lon, max_lon, diff_lon)

grid_lats = np.linspace(min_lat,max_lat,number_of_squares)
grid_lons = np.linspace(min_lon,max_lon,number_of_squares)

lat_centers = [(l1 + l2)/2 for l1, l2 in zip(grid_lats, grid_lats[1:])]
lon_centers = [(l1 + l2)/2 for l1, l2 in zip(grid_lons, grid_lons[1:])]

samples = []
for lat in lat_centers:
    for lon in lon_centers:
        samples.append((lat, lon))
cells += list(zip(samples, gen_fingerprints(samples, regs)))
print(cells[0][0][0])
print(cells[0][0][1])
print(np.mean(cells[0][1]))
len(cells)

C:\Users\guiga\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\guiga\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\guiga\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\guiga\AppData

SQUARE GRID GENERATOR
	Resolution: 20 m
-8.059338
-8.07757
-8.077090210526315
-34.906257289473686
127.69739471278955


C:\Users\guiga\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


361

In [44]:
# Voronoi
from scipy.spatial import Voronoi
points = [(lat, lon) for (lat, lon), _ in cells]
vor = Voronoi(points)

# Create map
map_ = folium.Map(location=[center_lat, center_lon],
                 zoom_start=13)

# Add voronoi cells
for i1, i2 in vor.ridge_vertices:
    if i1 != -1 and i2 != -1:
        p1 = vor.vertices[i1]
        p2 = vor.vertices[i2]
        folium.PolyLine([p1, p2],  color='black',weight=1, opacity=1).add_to(original_pts_map)

In [45]:
from operator import itemgetter, attrgetter

instance = df.iloc[[250]]
#In this Step, we will select the bts with the least pathloss, it means that the ms is probably near
#
bts1 = list(instance[['pathBTS1','taBTS1']].values)
bts2 = list(instance[['pathBTS2','taBTS2']].values)
bts3 = list(instance[['pathBTS3','taBTS3']].values)
bts4 = list(instance[['pathBTS4','taBTS4']].values)
bts5 = list(instance[['pathBTS5','taBTS5']].values)
bts6 = list(instance[['pathBTS6','taBTS6']].values)
#
#
bts_infos = [bts1,bts2,bts3,bts4,bts5,bts6]
#bts_infos.sort(key=lambda x: x[0])

lat_instance = instance['lat'].values[0]
lon_instance = instance['lon'].values[0]
print(lat_instance)
print(lon_instance)

#some_list = [(1,2),(2),(5),(3),(4),(7),(0)]
#some_list.sort()
#some_list
#bts_infos
bts_by_pathloss = [
       [bts1[0][0],bts1[0][1],0],
       [bts2[0][0],bts2[0][1],1],
       [bts3[0][0],bts3[0][1],2],
       [bts4[0][0],bts4[0][1],3],
       [bts5[0][0],bts5[0][1],4],
       [bts6[0][0],bts6[0][1],5]]
bts_sorted = sorted(bts_by_pathloss,key=itemgetter(0))
bts_sorted

-8.074223
-34.890274


[[95.19, 1.0, 0],
 [117.85666666666701, 0.0, 3],
 [120.29, 3.0, 2],
 [123.323333333333, 1.0, 4],
 [134.756666666667, 0.0, 1],
 [135.623333333333, 2.0, 5]]

In [46]:
time_advance = bts_sorted[0][1]
min_radius = (time_advance) * 550
radiusA = (time_advance + 1) * 550


bts_index = bts_sorted[0][2] #Select the nearest bts index
latA = df_btss['lat'][bts_index]
lonA = df_btss['lon'][bts_index]

print(latA)
print(lonA)


time_advance = bts_sorted[1][1]
min_radius = (time_advance) * 550
radiusB = (time_advance + 1) * 550


bts_index = bts_sorted[1][2] #Select the nearest bts index
#bts_index = 1
latB = df_btss['lat'][bts_index]
lonB = df_btss['lon'][bts_index]

print(latB)
print(lonB)

time_advance = bts_sorted[2][1]
min_radius = (time_advance) * 550
radiusC = (time_advance + 1) * 550


bts_index = bts_sorted[2][2] #Select the nearest bts index
latC = df_btss['lat'][bts_index]
lonC = df_btss['lon'][bts_index]

print(latC)
print(lonC)


#distance = get_distance_in_meters(lat_bts,lon_bts,-8.06500,-34.900)

#if(distance <=max_radius and distance > min_radius):
#    print('A distância é de {} metros Está dentro do range previsto'.format(distance))
#marker = folium.Circle(location=(lat_bts,lon_bts),color='blue',weight=1,radius=distance)
#marker.add_to(original_pts_map)

icon_p = folium.Icon(icon='mobile-phone', iconColor='white',color='black', prefix='fa')
marker = folium.Circle(location=(latA,lonA),color='green',weight=1,radius=radiusA, fill=True)
marker.add_to(original_pts_map)

marker = folium.Circle(location=(latB,lonB),color='red',weight=1,radius=radiusB, fill=True)
marker.add_to(original_pts_map)

marker = folium.Circle(location=(latC,lonC),color='blue',weight=1,radius=radiusC, fill=True)
#marker.add_to(original_pts_map)

marker = folium.Marker(location=(lat_instance,lon_instance),popup='teste',tooltip='celular',icon=icon_p)
marker.add_to(original_pts_map)

print(radiusA)
print(radiusB)
print(radiusC)

original_pts_map

-8.068361111
-34.892722221999996
-8.075916667000001
-34.8946111116
-8.076361111
-34.908
1100.0
550.0
2200.0


C:\Users\guiga\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:47: UserWarning: color argument of Icon should be one of: {'gray', 'darkgreen', 'lightblue', 'darkred', 'purple', 'pink', 'red', 'darkpurple', 'darkblue', 'lightgrayblack', 'green', 'orange', 'lightgreen', 'white', 'blue', 'cadetblue', 'lightred', 'beige'}.


Applying Trilateration
==

In [47]:
earthRadius = 6371
print(radiusA)
print(radiusB)
print(radiusC)

#distA = distA / 1000
#distB = distB / 1000
#distC = distC / 1000
distA = radiusA/1000
distB = radiusB/1000
distC = radiusC/1000

print(distA)
print(distB)
print(distC)

import math, numpy

xA = earthRadius * (math.cos( math.radians( latA )) * math.cos( math.radians( lonA )))
yA = earthRadius * (math.cos( math.radians( latA )) * math.sin( math.radians( lonA )))
zA = earthRadius * (math.sin(math.radians(latA)))                           

xB = earthRadius * (math.cos( math.radians( latB )) * math.cos( math.radians( lonB )))
yB = earthRadius * (math.cos( math.radians( latB )) * math.sin( math.radians( lonB )))
zB = earthRadius * (math.sin(math.radians(latB)))

xC = earthRadius * (math.cos( math.radians( latC )) * math.cos( math.radians( lonC )))
yC = earthRadius * (math.cos( math.radians( latC )) * math.sin( math.radians( lonC )))
zC = earthRadius * (math.sin(math.radians(latC)))

P1 = numpy.array([xA, yA, zA])
P2 = numpy.array([xB, yB, zB])
P3 = numpy.array([xC, yC, zC])

# A partir do algorítmo da wikipédia
#       Transforma o centro da espera P1 na origem
#       Tranforma o centro da esfera P2 no eixo x


ex = (P2 - P1) / (numpy.linalg.norm(P2 - P1))       # Vetor unitário da direção de P1 para P2
i = numpy.dot(ex, P3 - P1)                          # Magnitude do sinal da componente x, do vetor de P1 para P3 
ey = (P3 - P1 - i*ex) / (numpy.linalg.norm(P3 - P1 - i*ex))     # Vetor unitário da direção de y 
ez = numpy.cross(ex, ey)                                        # ez é o produto cartesiano entre (ex) x (ey)
 
d = numpy.linalg.norm(P2 - P1)         # distância entre os centros de P1 (0,0) e P2(d,0) 
j = numpy.dot(ey, P3 - P1)             # magnitude do sinal da componente y, do vetro de P1 para P3

x = ( pow(distA, 2) - pow(distB, 2) + pow(d, 2)) / (2*d)                          # x = ((r1^2 - r2^2 + d^2)^2) / (4d^2)
y = ((pow(distA, 2) - pow(distC, 2) + pow(i, 2) + pow(j,2)) / (2*j)) - ((i/j)*x)  # y = (r1^2 - r3^2 + i^2 +j^2)/2j - (i/j)x
z = numpy.sqrt(pow(distA, 2) - pow(x, 2) - pow(y, 2))                             # z = +-sqrt(r1^2 - x^2 - y^2)

# TriPt é um array com os pontos x,y e z em ECEF, ponto de trilateração utilizado para fazer a conversão para lat/long

triPt = P1 + x*ex + y*ey + z*ez

# Converte de ECEF para lat/long e converte em graus

lat = math.degrees(math.asin(triPt[2] / earthRadius))
lon = math.degrees(math.atan2(triPt[1], triPt[0]))

print (lat, lon)
icon_predicted = folium.Icon(color='white',icon_color='blue')
marker = folium.Marker(location=(lat,lon),popup='Predicted Point',tooltip='Ponto Predito',icon=icon_predicted)
marker.add_to(original_pts_map)

1100.0
550.0
2200.0
1.1
0.55
2.2
nan nan


C:\Users\guiga\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in sqrt


ValueError: Location values cannot contain NaNs.

In [54]:
original_pts_map